In [ ]:
import os
import getpass
from google.colab import userdata
import langchain

In [ ]:
!pip install langchain
!pip install google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.1 MB/s eta 0:00:00


In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [ ]:
import google.generativeai as genai

In [ ]:
!pip install --upgrade langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat = ChatGoogleGenerativeAI(model = 'gemini-pro', temperature = 0.9)
response = chat.invoke("What is capital of france")
print(response.content)

Paris


**CommaSeparatedList Output_Parser**

In [ ]:
from langchain import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
print(output_parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
prompt = PromptTemplate(
    template="List five  {subject}.\n{format_instruction}",
    input_variables = ['subject'],
    partial_variables = {'format_instruction': output_parser.get_format_instructions()}
)

In [ ]:
format_prompt = prompt.format(subject = 'Ice cream flavours')
print(format_prompt)

List fiveIce cream flavours


In [ ]:
chat_model = chat.invoke(format_prompt)
print(chat_model.content)

1. Vanilla
2. Chocolate
3. Strawberry
4. Pistachio
5. Mint


In [ ]:
output_parser.parse(chat_model.content)

['1. Vanilla\n2. Chocolate\n3. Strawberry\n4. Pistachio\n5. Mint']

**DateTime Output_Parse**


In [ ]:
from langchain.output_parsers import DatetimeOutputParser
output_parser = DatetimeOutputParser()
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 14-02-07T02:29:37.859179Z, 921-02-21T05:52:15.515038Z, 971-11-26T19:22:46.404446Z

Return ONLY this string, no other words!


In [ ]:
prompt = PromptTemplate(
    template = "Answer the followng question : {question}",
    input_variable = ['question'],
    partial_variables = {'format_instruction': output_parser.get_format_instructions()}
)

In [ ]:
final_prompot = prompt.format(question = "When is Lunar New Year of 2024?")
print(final_prompot)

Answer the followng question : When is Lunar New Year of 2024?


In [ ]:
chat_model = chat.invoke(final_prompot)
print(chat_model)

content='January 22, 2024' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-31158633-4cda-461f-b6b0-c78386d013f8-0' usage_metadata={'input_tokens': 19, 'output_tokens': 10, 'total_tokens': 29}


**JSON Output_Parser**


In [ ]:
from langchain_core.output_parsers import JsonOutputParser
parse = JsonOutputParser()
from langchain_core.pydantic_v1 import BaseModel , Field

In [ ]:
class joke(BaseModel):
    setup: str = Field(Description = "question to set up joke")
    punchline: str = Field(Description = "answer to resolve the joke")

parser = JsonOutputParser(pydantic_object = joke)


In [ ]:
prompt = PromptTemplate(
    template = "Answer the query: {query}.\n{format_instructions}",
    input_variables = ['query'],
    partial_variables = {'format_instructions': parser.get_format_instructions}
)

In [ ]:
final_prompt = prompt.format(query = "Tell me about Chicken joke")

chat_model = chat.invoke(final_prompt)
print(chat_model.content)

{"setup": "What do you call a chicken that can't lay eggs?", "punchline": "An egg-stremely lazy chicken!"}
